# Animal Detection


## Project Overview

The goal of this project is to build an image classification model that can accurately identify different types of animals from input images. We'll be using the animals detection images dataset, which contains over 3,000 labeled images of 5 different animal classes: cats, dogs, elephants, horses, and humans.

Our final deliverable will be a Flask-based web application that allows users to upload an image and get a prediction of the most likely animal class that the image contains.


## Project Steps

1. Data exploration and preprocessing: We'll start by exploring the dataset and performing some basic preprocessing steps. This will include visualizing the images, checking for class balance, and splitting the dataset into training, validation, and test sets.

2. Feature extraction and model selection: Next, we'll use transfer learning to extract features from the pre-trained VGG-16 model, which has been shown to be effective for image classification tasks. We'll also experiment with other pre-trained models such as ResNet, Inception, and EfficientNet, and select the one that performs the best on our validation set.

3. Model training and evaluation: We'll then train the selected model on our training set and evaluate its performance on our validation set. We'll use techniques such as data augmentation, regularization, and hyperparameter tuning to improve the model's accuracy and prevent overfitting.

4. Model deployment and Flask integration: Once we have a trained and validated model, we'll deploy it as a REST API using Flask. We'll create a simple web interface that allows users to upload an image and get a prediction of the most likely animal class that the image contains. We'll also implement error handling and user feedback to make the interface more user-friendly.


## Project Implementation


### Data Exploration and Preprocessing


In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths to dataset and subdirectories
base_path = "/path/to/animals_detection_images"
train_path = os.path.join(base_path, "train")
val_path = os.path.join(base_path, "val")
test_path = os.path.join(base_path, "test")

# Define batch size and image size
batch_size = 32
img_size = (224, 224)

# Create image data generators with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators for training, validation, and test sets
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical"
)

# Plot some example images
def plot_examples(generator, n_examples=6):
    classes = list(generator.class_indices.keys())
    for example in range(n_examples):
        x, y = generator.next()
        plt.figure(figsize=(10, 10))
    for i in range(batch_size):
        plt.subplot(3, 3, i+1)
        plt.imshow(x[i])
        plt.title(classes[np.argmax(y[i])])
        plt.axis("off")
        plt.tight_layout()
        plt.show()

plot_examples(train_generator)

### Feature Extraction and Model Selection

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Define pre-trained model and its input shape
base_model = VGG16(weights="imagenet", include_top=False, input_shape=img_size+(3,))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(5, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=x)

# Compile model with categorical crossentropy loss and Adam optimizer
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Print model summary
model.summary()

### Model Training and Evaluation

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks for model saving and early stopping
checkpoint = ModelCheckpoint("animals_detection_model.h5", save_best_only=True, verbose=1)
early_stopping = EarlyStopping(patience=5, restore_best_weights=True, verbose=1)

# Train model on training set and validate on validation set
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping]
)

# Evaluate model on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")


### Model Deployment and Flask Integration

In [ ]:
import io
from PIL import Image
from flask import Flask, jsonify, request
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Define paths to model and label files
model_path = "animals_detection_model.h5"
label_path = "labels.txt"

# Load model and labels
model = load_model(model_path)
with open(label_path, "r") as f:
    labels = [line strip() for line in f]

# Define Flask app
app = Flask(name)

# Define predict function
def predict(image):
    # Preprocess input image
    image = image.convert("RGB").resize(img_size)
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)

# Get model prediction
prediction = model.predict(image)[0]
label = labels[np.argmax(prediction)]
confidence = float(prediction[np.argmax(prediction)])

return {"label": label, "confidence": confidence}

In [ ]:
# Define Flask endpoints
@app.route("/")
def index():
return """
<h1>Animals Detection</h1>
<form method="POST" action="/predict" enctype="multipart/form-data">
<input type="file" name="file"><br><br>
<input type="submit" value="Predict">
</form>
"""

@app.route("/predict", methods=["POST"])
def predict_endpoint():
# Get input image from request
file = request.files["file"]
image = Image.open(io.BytesIO(file.read()))

# Get prediction
prediction = predict(image)

# Return prediction as JSON
return jsonify(prediction)

# Run Flask app
if name == "main":
app.run(debug=True)


This code creates a Flask app and defines a `predict()` function that preprocesses an input image and returns a prediction. It then defines two Flask endpoints: `/` for a simple web interface that allows users to upload an image and get a prediction, and `/predict` for the API endpoint that calls the `predict()` function and returns the prediction as a JSON object. Finally, it runs the Flask app in debug mode.

Note that we'll also need to create a `labels.txt` file that contains the animal class labels, one per line.

With these steps, we have created a complete data science project that includes data preprocessing, model selection and training, and model deployment and integration with Flask. We can now run the Flask app, upload an image of an animal, and get a prediction of its most likely class.
